In [1]:
import pandas as pd

df = pd.DataFrame({
    "timestamp": [
        "2025-11-25 09:30:00",
        "2025-11-25 09:31:00",
        "2025-11-25 09:32:00",
        "2025-11-25 09:33:00",
        "2025-11-25 09:34:00",
    ],
    "symbol": ["AAPL", "AAPL", "AAPL", "AAPL", "AAPL"],
    "last": [99.8, 100.3, 101.1, 102.0, 101.6],
    "bid":  [99.5, 100.2, 101.0, 101.9, 101.5],
    "ask":  [100.0, 100.4, 101.2, 102.1, 101.7],
    "volume": [1000, 1200, 1100, 1300, 900]
})

df


,timestamp,symbol,last,bid,ask,volume
0,2025-11-25 09:30:00,AAPL,99.8,99.5,100.0,1000
1,2025-11-25 09:31:00,AAPL,100.3,100.2,100.4,1200
2,2025-11-25 09:32:00,AAPL,101.1,101.0,101.2,1100
3,2025-11-25 09:33:00,AAPL,102.0,101.9,102.1,1300
4,2025-11-25 09:34:00,AAPL,101.6,101.5,101.7,900


In [2]:
df.to_csv("demo_intraday.csv", index=False)


In [1]:
import pandas as pd

timestamps = [
    "2025-11-25 09:30:00",
    "2025-11-25 09:31:00",
    "2025-11-25 09:32:00",
    "2025-11-25 09:33:00",
    "2025-11-25 09:34:00",
]

# Build a small 2-symbol dataset: AAPL and MSFT
rows = []

# AAPL path (similar to your old one)
aapl_prices = [
    (99.8,  99.5, 100.0, 1000),
    (100.3, 100.2, 100.4, 1200),
    (101.1, 101.0, 101.2, 1100),
    (102.0, 101.9, 102.1, 1300),
    (101.6, 101.5, 101.7, 900),
]

for ts, (last, bid, ask, vol) in zip(timestamps, aapl_prices):
    rows.append({
        "timestamp": ts,
        "symbol": "AAPL",
        "last": last,
        "bid": bid,
        "ask": ask,
        "volume": vol,
    })

# MSFT path – just make up something slightly different
msft_prices = [
    (250.0, 249.8, 250.2, 1500),
    (251.0, 250.9, 251.2, 1600),
    (252.5, 252.4, 252.7, 1550),
    (251.8, 251.7, 252.0, 1400),
    (253.0, 252.9, 253.2, 1450),
]

for ts, (last, bid, ask, vol) in zip(timestamps, msft_prices):
    rows.append({
        "timestamp": ts,
        "symbol": "MSFT",
        "last": last,
        "bid": bid,
        "ask": ask,
        "volume": vol,
    })

df_multi = pd.DataFrame(rows)

# Sort by time then symbol just to be neat
df_multi = df_multi.sort_values(["timestamp", "symbol"]).reset_index(drop=True)

df_multi


,timestamp,symbol,last,bid,ask,volume
0,2025-11-25 09:30:00,AAPL,99.8,99.5,100.0,1000
1,2025-11-25 09:30:00,MSFT,250.0,249.8,250.2,1500
2,2025-11-25 09:31:00,AAPL,100.3,100.2,100.4,1200
3,2025-11-25 09:31:00,MSFT,251.0,250.9,251.2,1600
4,2025-11-25 09:32:00,AAPL,101.1,101.0,101.2,1100
5,2025-11-25 09:32:00,MSFT,252.5,252.4,252.7,1550
6,2025-11-25 09:33:00,AAPL,102.0,101.9,102.1,1300
7,2025-11-25 09:33:00,MSFT,251.8,251.7,252.0,1400
8,2025-11-25 09:34:00,AAPL,101.6,101.5,101.7,900
9,2025-11-25 09:34:00,MSFT,253.0,252.9,253.2,1450


In [2]:
df_multi.to_csv("demo_intraday_multi.csv", index=False)


In [6]:
import yfinance as yf
import pandas as pd

symbols = ["AAPL", "MSFT"]
start = "2024-10-01"
end = "2024-10-10"

dfs = []

for sym in symbols:
    data = yf.download(sym, start=start, end=end, interval="1m")
    data = data.dropna()
    
    df = data.reset_index()

    # Normalize timestamp column
    if "Datetime" in df.columns:
        df = df.rename(columns={"Datetime": "timestamp"})
    elif "Date" in df.columns:
        df = df.rename(columns={"Date": "timestamp"})
    else:
        df.columns = ["timestamp"] + list(df.columns[1:])
    
    df["symbol"] = sym

    # our engine needs: timestamp, symbol, bid, ask, last, volume
    out = pd.DataFrame()
    out["timestamp"] = df["timestamp"]
    out["symbol"] = df["symbol"]
    out["last"] = df["Close"]
    out["volume"] = df["Volume"]

    # approximate bid/ask
    out["bid"] = out["last"] * 0.9995
    out["ask"] = out["last"] * 1.0005

    dfs.append(out)

df_all = pd.concat(dfs).sort_values(["timestamp", "symbol"]).reset_index(drop=True)
df_all.to_csv("intraday_multi.csv", index=False)


C:\Users\kvssr\AppData\Local\Temp\ipykernel_9256\932669544.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(sym, start=start, end=end, interval="1m")
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('possibly delisted; no price data found  (1m 2024-10-01 -> 2024-10-10) (Yahoo error = "1m data not available for startTime=1727755200 and endTime=1728532800. Only 8 days worth of 1m granularity data are allowed to be fetched per request.")')
C:\Users\kvssr\AppData\Local\Temp\ipykernel_9256\932669544.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(sym, start=start, end=end, interval="1m")
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MSFT']: YFPricesMissingError('possibly delisted; no price data found  (1m 2024-10-01 -> 2024-10-10) (Yahoo error = "1m data not avail

In [8]:
import yfinance as yf
import pandas as pd

symbols = ["AAPL", "MSFT"]

dfs = []

for sym in symbols:
    data = yf.download(sym, period="5d", interval="1m")  # <= 5 days of 1m data
    data = data.dropna()
    print(sym, data.shape)  # sanity check

    df = data.reset_index()

    # normalize timestamp column
    if "Datetime" in df.columns:
        df = df.rename(columns={"Datetime": "timestamp"})
    elif "Date" in df.columns:
        df = df.rename(columns={"Date": "timestamp"})
    else:
        df.columns = ["timestamp"] + list(df.columns[1:])

    df["symbol"] = sym

    out = pd.DataFrame()
    out["timestamp"] = df["timestamp"]
    out["symbol"] = df["symbol"]
    out["last"] = df["Close"]
    out["volume"] = df["Volume"]
    out["bid"] = out["last"] * 0.9995
    out["ask"] = out["last"] * 1.0005

    dfs.append(out)

df_all = pd.concat(dfs).sort_values(["timestamp", "symbol"]).reset_index(drop=True)
print("Total rows:", df_all.shape[0])

df_all.to_csv("intraday_multi.csv", index=False)


C:\Users\kvssr\AppData\Local\Temp\ipykernel_9256\3232094177.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(sym, period="5d", interval="1m")  # <= 5 days of 1m data
[*********************100%***********************]  1 of 1 completed
C:\Users\kvssr\AppData\Local\Temp\ipykernel_9256\3232094177.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(sym, period="5d", interval="1m")  # <= 5 days of 1m data
[*********************100%***********************]  1 of 1 completed

AAPL (1950, 5)
MSFT (1950, 5)
Total rows: 3900
